In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Create a DataFrame with the provided rows
df=pd.read_csv("comwast_severe.csv")
#df=df.dropna(subset=['over_r'], inplace=True)

df=df.drop(['rural_severe_r','rural_severe_weighted','severe_Weighted'],axis=1)
df.dropna(subset=['severe_r'], inplace=True)


df

,ISO3Code,UNRegion,WB_Latest,age,sex,mothers_education,severe_r
0,AFG,Asia,Low Income,month_0_5,male,none,1.697622
1,AFG,Asia,Low Income,month_0_5,male,primary,1.805691
2,AFG,Asia,Low Income,month_0_5,male,none-primary,1.595920
3,AFG,Asia,Low Income,month_0_5,male,secondary,3.220327
4,AFG,Asia,Low Income,month_0_5,male,higher,2.177415
...,...,...,...,...,...,...,...
13495,ZWE,Africa,Lower Middle Income,month_24_59,female,primary,0.150285
13496,ZWE,Africa,Lower Middle Income,month_24_59,female,none-primary,0.204156
13497,ZWE,Africa,Lower Middle Income,month_24_59,female,secondary,0.296027
13498,ZWE,Africa,Lower Middle Income,month_24_59,female,higher,0.119165


In [3]:
df=df.drop(df[df['age']=='month_0_23 '].index)
#df=df.drop(['ISO3Code'], inplace=True)


df=df.drop(df[df['age']=='month_24_59 '].index)
df=df.drop(df[df['mothers_education']=='none'].index)
df=df.drop(df[df['mothers_education']=='higher'].index)
df = df.reset_index(drop=True)
df=df.rename(columns={'severe_r':'severe_wasting_estimate'})
df.columns.tolist()
df.dropna(subset=['severe_wasting_estimate'], inplace=True)
df.to_csv("severe_wasting_r.csv",index=False)
df

,ISO3Code,UNRegion,WB_Latest,age,sex,mothers_education,severe_wasting_estimate
0,AFG,Asia,Low Income,month_0_5,male,primary,1.805691
1,AFG,Asia,Low Income,month_0_5,male,none-primary,1.595920
2,AFG,Asia,Low Income,month_0_5,male,secondary,3.220327
3,AFG,Asia,Low Income,month_0_5,male,higher_secondary,2.375922
4,AFG,Asia,Low Income,month_6_11,male,primary,1.333480
...,...,...,...,...,...,...,...
3405,ZWE,Africa,Lower Middle Income,month_36_47,female,higher_secondary,0.346830
3406,ZWE,Africa,Lower Middle Income,month_48_59,female,primary,0.150733
3407,ZWE,Africa,Lower Middle Income,month_48_59,female,none-primary,0.230025
3408,ZWE,Africa,Lower Middle Income,month_48_59,female,secondary,0.340325


In [4]:
df=pd.read_csv("overweight_r.csv")
encoded_df = pd.get_dummies(df[['ISO3Code','UNRegion', 'WB_Latest', 'age', 'sex', 'mothers_education']])

# Concatenate the encoded features with the target variable
encoded_df = pd.concat([encoded_df, df['Overweight_estimate']], axis=1)
df=encoded_df
df = df.replace({True: 1, False: 0})


df

,ISO3Code_AFG,ISO3Code_AGO,ISO3Code_ALB,ISO3Code_ARM,ISO3Code_BEN,ISO3Code_BGD,ISO3Code_BIH,ISO3Code_BLR,ISO3Code_BLZ,ISO3Code_BRB,...,age_month_36_47,age_month_48_59,age_month_6_11,sex_ female,sex_ male,mothers_education_higher_secondary,mothers_education_none-primary,mothers_education_primary,mothers_education_secondary,Overweight_estimate
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,5.119386
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,4.277103
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,7.965074
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,6.295756
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,3.620351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,2.527198
3358,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,1.805866
3359,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,1.826800
3360,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,2.301626


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
data = df


# Split the data into features and target variable
X = data.drop('Overweight_estimate', axis=1)
y = data['Overweight_estimate']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree regressor
regressor = DecisionTreeRegressor(random_state=42)

# Fit the regressor to the training data
regressor.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = regressor.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
from sklearn.metrics import r2_score

r_squared = r2_score(y_test, y_pred)
print("R-squared:", r_squared)
print('Mean Squared Error:', mse)



R-squared: 0.9675670785722413
Mean Squared Error: 0.5637854996981515


In [6]:
da=pd.read_csv("Joint_estimates.csv")
da = da.round(2)
da.to_csv("Joint_estimates.csv",index=True)
da

,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,...,mothers_education,stunt_weighted,stunt_r,over_weighted,over_r,Wasting_Weighted,Wasting_r,severe_Weighted,severe_r,Status
0,0,0,0,0,0,0,0,0,0,0,...,none,16820.0,39.16,17281.8,4.49,17281.8,5.47,17281.8,1.70,stunting
1,1,1,1,1,1,1,1,1,1,1,...,primary,2298.7,23.70,2333.6,5.12,2333.6,7.03,2333.6,1.81,stunting
2,2,2,2,2,2,2,2,2,2,2,...,none-primary,18150.6,38.41,18623.0,4.28,18623.0,5.45,18623.0,1.60,stunting
3,3,3,3,3,3,3,3,3,3,3,...,secondary,1621.2,20.11,1649.5,7.97,1649.5,8.23,1649.5,3.22,stunting
4,4,4,4,4,4,4,4,4,4,4,...,higher,2173.8,18.48,2262.1,6.34,2262.1,6.48,2262.1,2.18,stunting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5913,5913,5913,5913,5913,5913,5913,5913,5913,5913,5913,...,primary,3737.8,25.67,3740.7,1.90,3740.7,2.55,3740.7,0.15,stunting
5914,5914,5914,5914,5914,5914,5914,5914,5914,5914,5914,...,none-primary,3877.5,25.62,3880.7,1.91,3880.7,2.62,3880.7,0.20,stunting
5915,5915,5915,5915,5915,5915,5915,5915,5915,5915,5915,...,secondary,5363.6,22.27,5365.9,2.25,5365.9,2.52,5365.9,0.30,stunting
5916,5916,5916,5916,5916,5916,5916,5916,5916,5916,5916,...,higher,2301.3,20.64,2300.6,2.02,2300.6,2.08,2300.6,0.12,stunting


In [7]:
import pandas as pd

df=pd.read_csv("overweight_r.csv")


#FINDING MAX AND MIN
p=df['Overweight_estimate'].max()
q=df['Overweight_estimate'].min()

print(p)
print(q)
x=p-q
print(x)

20.9
0.7408637873754153
20.159136212624585
